In [1]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
import pandas as pd
import numpy as np
import re
import math
import numpy as np
from gensim.models import Word2Vec

In [17]:
from sklearn.decomposition import PCA

# 차원을 줄여 보도록 하자!

In [ ]:
#https://archive.ics.uci.edu/ml/datasets/Online%20Retail
# df= pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
# df.to_excel('./data/retail_data.xlsx')

In [18]:
df = pd.read_excel('./data/retail_data.xlsx')

In [19]:
stop_words = set(stopwords.words('english')) 
def str_filter(x):
    if type(x) == str:
        
        x = x.strip(' ').lower().replace('-', ' ').replace('/' , ' ')
        x = re.sub('[0-9]', '', x)
        x = re.sub('\b\w{1}\b', '', x)
        
        stop_words = set(stopwords.words('english'))         
        filtered_product_name_set = set([w for w in x.split(' ') if not w in stop_words] )
        return filtered_product_name_set
    else:
        return None


df['Description_filtered'] = df['Description'].apply(lambda x : str_filter(x))


In [23]:
df['Description_filtered'].values

array([{'light', 'heart', 'white', 'hanging', 'holder'},
       {'white', 'lantern', 'metal'},
       {'hearts', 'cream', 'coat', 'cupid', 'hanger'}, ...,
       {'dolly', 'cutlery', 'childrens', 'girl'},
       {'cutlery', 'parade', 'circus', 'childrens'},
       {'', 'piece', 'retrospot', 'baking', 'set'}], dtype=object)

In [28]:
def word_to_vector_dict(list_of_sets):
    dic = dict()
    i = 0
    for each_set in list_of_sets:
        if each_set != None:
            for each_word in each_set:
                if each_word not in dic.keys():
                    i +=1
                    dic[each_word] =i
    return dic, len(dic.keys())

    

In [29]:
dic, dic_len = word_to_vector_dict(df['Description_filtered'].values)

In [33]:
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Description_filtered
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,"{light, heart, white, hanging, holder}"
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{white, lantern, metal}"
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,"{hearts, cream, coat, cupid, hanger}"
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{union, water, flag, hot, knitted, bottle}"
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,"{red, woolly, white, hottie, heart.}"


In [55]:
vector_dic = dict()
for product_name, product_word in zip(df['Description'], df['Description_filtered']):
    if product_word != None:
        temp_array = np.zeros(dic_len + 1)
        for each_word in product_word:
            if each_word != None:
                temp_array[dic[each_word]] = 1
        vector_dic[product_name] = temp_array


In [53]:
vector_dic

{'WHITE HANGING HEART T-LIGHT HOLDER': array([0., 1., 1., ..., 0., 0., 0.]),
 'WHITE METAL LANTERN': array([0., 0., 0., ..., 0., 0., 0.]),
 'CREAM CUPID HEARTS COAT HANGER': array([0., 0., 0., ..., 0., 0., 0.]),
 'KNITTED UNION FLAG HOT WATER BOTTLE': array([0., 0., 0., ..., 0., 0., 0.]),
 'RED WOOLLY HOTTIE WHITE HEART.': array([0., 0., 0., ..., 0., 0., 0.]),
 'SET 7 BABUSHKA NESTING BOXES': array([0., 0., 0., ..., 0., 0., 0.]),
 'GLASS STAR FROSTED T-LIGHT HOLDER': array([0., 1., 0., ..., 0., 0., 0.]),
 'HAND WARMER UNION JACK': array([0., 0., 0., ..., 0., 0., 0.]),
 'HAND WARMER RED POLKA DOT': array([0., 0., 0., ..., 0., 0., 0.]),
 'ASSORTED COLOUR BIRD ORNAMENT': array([0., 0., 0., ..., 0., 0., 0.]),
 "POPPY'S PLAYHOUSE BEDROOM ": array([0., 0., 0., ..., 0., 0., 0.]),
 "POPPY'S PLAYHOUSE KITCHEN": array([0., 0., 0., ..., 0., 0., 0.]),
 'FELTCRAFT PRINCESS CHARLOTTE DOLL': array([0., 0., 0., ..., 0., 0., 0.]),
 'IVORY KNITTED MUG COSY ': array([0., 0., 0., ..., 0., 0., 0.]),
 'BOX 

In [75]:
from sklearn.decomposition import PCA

word_lis = []
vector_lis = []
for k, v in vector_dic.items():
    word_lis.append(k)
    vector_lis.append(v)
    

pca = PCA(n_components=2)
pca.fit(vector_lis)  

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [66]:
PCA_vec = PCA(n_components=2).fit_transform(X)

In [84]:
def cosine_distance(org, comp):
    A = np.array(org)
    B = np.array(comp)
    up = np.sum(A*B)
    down = np.sqrt(np.sum(np.square(A))) * np.sqrt(np.sum(np.square(B)))
    score = round(up/down, 2)
    return score


In [86]:
org = vector_lis[0]
org_name = word_lis[0]
for product_name, comp in zip(word_lis, vector_lis):
    if cosine_distance(org, comp) > 0.5:
        print(product_name)
    

WHITE HANGING HEART T-LIGHT HOLDER
RED HANGING HEART T-LIGHT HOLDER
COLOUR GLASS T-LIGHT HOLDER HANGING
HANGING HEART ZINC T-LIGHT HOLDER
SINGLE HEART ZINC T-LIGHT HOLDER
SILVER HANGING T-LIGHT HOLDER
HEART T-LIGHT HOLDER 
T-LIGHT HOLDER HANGING LACE
HANGING JAM JAR T-LIGHT HOLDER
GLASS HEART T-LIGHT HOLDER 
ZINC HEART LATTICE T-LIGHT HOLDER
HANGING BAUBLE T-LIGHT HOLDER SMALL
HANGING BAUBLE T-LIGHT HOLDER LARGE
WHITE TALL PORCELAIN T-LIGHT HOLDER
HEART STRING MEMO HOLDER HANGING
T-LIGHT HOLDER WHITE LACE
PERIWINKLE T-LIGHT HOLDER
HANGING HEART JAR T-LIGHT HOLDER
T-LIGHT HOLDER SWEETHEART HANGING
WHITE HANGING BEADS CANDLE HOLDER
HEART T-LIGHT HOLDER
FOLK ART METAL HEART T-LIGHT HOLDER
HANGING HEART BASKET
HANGING RIDGE GLASS T-LIGHT HOLDER
HANGING HEART WITH BELL
ZINC HEART FLOWER T-LIGHT HOLDER
ZINC  HEART T-LIGHT HOLDER
HEART T-LIGHT HOLDER WILLIE WINKIE
T-LIGHT HOLDER SILVER HEART HANDLE
HANGING HEART BONHEUR
HEART TRELLISTRIPLE T-LIGHT HOLDER
HANGING  BUTTERFLY T-LIGHT HOLDER
HEAR

In [107]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=10, random_state=0).fit(PCA_vec)


In [108]:
predict_dic = {x : [] for x in range(10)}
for each_vec, product_name in zip(PCA_vec, word_lis):
    result = kmeans.predict(each_vec.reshape(-1, 2))
    predict_dic[result[0]].append(product_name)
    

In [109]:
for k,v in predict_dic.items():
    print(len(v))

1707
441
285
296
30
83
813
219
340
8
